!!! Logging with huggingface tokens using the command

```bash
huggingface-cli login --token <YOUR HF TOKEN>
```

In [ ]:
from accelerate.utils import merge_fsdp_weights

# FSDP weight directory to merge
FSDP_WEIGHT_DIR = "./outputs/checkpoint-5760/pytorch_model_fsdp_0"
# Directory to save the merged weights
MERGED_WEIGHT_DIR = "./model_merged"

merge_fsdp_weights(FSDP_WEIGHT_DIR, MERGED_WEIGHT_DIR, safe_serialization=True)

In [ ]:
import torch
from transformers import AutoModelForCausalLM
from huggingface_hub import snapshot_download

# Download the model from the Hugging Face Hub
snapshot_download(repo_id="jan-hq/llama-3-sound-instruct-cp", local_dir="./llama-3-sound-instruct-cp")

# Load the current model weights
model = AutoModelForCausalLM.from_pretrained(
    "./llama-3-sound-instruct-cp",
    torch_dtype=torch.bfloat16,
    use_cache=False,
    device_map="cpu"
)

In [ ]:
# Load the merged FSDP weights
from safetensors.torch import load_file
new_merged_model_state = load_file("./model_merged/model.safetensors")

In [ ]:
# Load the model with the merged FSDP weights
model.load_state_dict(new_merged_model_state)
model.push_to_hub("jan-hq/llama-3-sound-instruct-cp")